In [4]:
'''
Note: this is just a template for PA 1 and the code is for references only.
Feel free to design the pipeline of the *main* function. However, one should keep
the interfaces for the other functions unchanged. Change the returned values of
these functions so that they are consistent with the assignment instructions.
In general, one will only need to add the code below the TO-DO statements to
finish the assignment. Additional import statements can be included when needed.

For the kNN classifier, one could use existing libraries to compute the pairwise
Euclidean distances between the test and training data, as for-loops in Python
are pretty slow. Other than that, the designs of all functions should be your
original work.
'''

import csv
import numpy as np

def compute_accuracy(test_y, pred_y):

    # TO-DO: add your code here

    return None

def test_knn(train_x, train_y, test_x, num_nn):

    # TO-DO: add your code here

    return None

def test_pocket(w, test_x):

    # TO-DO: add your code here

    return None

def train_pocket(train_x, train_y, num_iters):

    # TO-DO: add your code here

    return None

def get_id():

    return 'tuk01401'

def main():

    # Read the data file
    szDatasetPath = './letter-recognition.data' # Put this file in the same place as this script
    listClasses = []
    listAttrs = []
    with open(szDatasetPath) as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')
        for row in csvReader:
            listClasses.append(row[0])
            listAttrs.append(list(map(float, row[1:])))

    # Generate the mapping from class name to integer IDs
    mapCls2Int = dict([(y, x) for x, y in enumerate(sorted(set(listClasses)))])

    # Store the dataset with numpy array
    dataX = np.array(listAttrs)
    dataY = np.array([mapCls2Int[cls] for cls in listClasses])

    # Split the dataset as the training set and test set
    nNumTrainingExamples = 15000
    trainX = dataX[:nNumTrainingExamples, :]
    trainY = dataY[:nNumTrainingExamples]
    testX = dataX[nNumTrainingExamples:, :]
    testY = dataY[nNumTrainingExamples:]

    # TO-DO: add your code here

    return None

if __name__ == "__main__":
    main()


In [5]:
import csv
import numpy as np

# Read the data file
szDatasetPath = './letter-recognition.data' # Put this file in the same place as this script
listClasses = []
listAttrs = []
with open(szDatasetPath) as csvFile:
    csvReader = csv.reader(csvFile, delimiter=',')
    for row in csvReader:
        listClasses.append(row[0])
        listAttrs.append(list(map(float, row[1:])))

# Generate the mapping from class name to integer IDs
mapCls2Int = dict([(y, x) for x, y in enumerate(sorted(set(listClasses)))])

# Store the dataset with numpy array
dataX = np.array(listAttrs)
dataY = np.array([mapCls2Int[cls] for cls in listClasses])

# Split the dataset as the training set and test set
nNumTrainingExamples = 15000
trainX = dataX[:nNumTrainingExamples, :]
trainY = dataY[:nNumTrainingExamples]
testX = dataX[nNumTrainingExamples:, :]
testY = dataY[nNumTrainingExamples:]

# TO-DO: add your code here

In [6]:
testX.shape

(5000, 16)

**This is the KNN operator**

In [41]:
#test_knn(train_x, train_y, test_x, num_nn)
num_nn = 1

#TODO can we assume the matrix shapes will match?
neighb_dict = {}
squared = 0
pred_labels = np.empty([testX.shape[0],], dtype=int)

#Go through every test point
for i in range(0, 50):
    #Go through every training point
    for j in range(0, trainX.shape[0]):
        #Find the disctance between the test point and training point
        for k in range(0, trainX.shape[1]):
            squared += ((testX[i,k] - trainX[j,k])**2)
        neighb_dict[j] = (squared**.5)
        squared = 0
    #Sort the dictionary by the values not keys (taken from most common stackoverflow answer) 
    dict_sorted = {k: v for k, v in sorted(neighb_dict.items(), key=lambda item: item[1])}
    
    keys_list = list(dict_sorted)
    nearest_labels = {}
    
    for l in range(0, num_nn):
        if trainY[keys_list[l]] in nearest_labels:
            nearest_labels[trainY[keys_list[l]]] += 1
        else:
            nearest_labels[trainY[keys_list[l]]] = 1
    #TODO Can I assume the top label is "random" from the sorting in the case of ties
    labels_sorted = {k: v for k, v in sorted(nearest_labels.items(), key=lambda item: item[1], reverse=True)}
    
    #TODO I think these are coming out as floats
    pred_labels[i] = list(labels_sorted.keys())[0]

#print(pred_labels)

**Training Pocket**

In [60]:
import random

def compute_accuracy(test_y, pred_x):

    # Returns an array with true at every match
    correct_labels = (test_y == pred_y)

    count = 0
    
    for i in range(0, test_y.shape[0]):
        if (correct_labels == True):
            count += 1
    accuracy = count / test_y.shape[0]
    
    return accuracy

def test_pocket(w, test_x):

    # Add the extra dimension to the testing features to account for the b
    extra_dim = np.ones((test_X.shape[0],1), dtype=int)
    test_X_mod = np.append(test_X, extra_dim, axis=1)
    
    # Create arrays for the guesses of each letter, and the final prediction
    dot_prods = np.empty(26,)
    preds = np.empty(test_x.shape[0])

    # Go through each testing point and test it for every letter.  Prediction is the highest scoring
    for i in range (0, test_X_mod.shape[0]):
        for j in range (0, 26):
            dot_prods[j] = np.dot(w[j], test_X_mod[i])
        preds[i] = np.argnax(dot_prods)
        
    return preds

#Add the extra dimension here to make it easier to get the dot product later (weights have b included)
extra_dim = np.ones((trainX.shape[0],1), dtype=int)
trainX_mod = np.append(trainX, extra_dim, axis=1)

#A vector of weights for each dimension and the 1 extra, and k labels worth of weights
weights = np.zeros([26,trainX_mod.shape[1]])

num_iters = 1
preds = np.empty_like(trainY)
misclassified_list = []

for i in range(0, num_iters):
    for j in range(0, 26):
        for k in range(0, trainX.shape[0]):
            preds[k] = np.dot(weights[j], trainX_mod[k])
            if (((trainY[k] == j) and (preds[k] <= 0)) or ((trainY[k] != j) and (preds[k] > 0))):
                misclassified_list.append(k)
        point_to_update = random.choice(misclassified_list)
        if (trainY[point_to_update] == j):
            weights[j] += trainX_mod[point_to_update]
        else:
            weights[j] -= trainX_mod[point_to_update] 
        

In [54]:
print(misclassified_list)

[7, 78, 118, 130, 134, 144, 147, 157, 163, 216, 232, 234, 289, 295, 304, 313, 327, 337, 415, 473, 483, 502, 517, 608, 638, 675, 686, 719, 732, 749, 769, 789, 790, 813, 892, 944, 958, 991, 1014, 1023, 1027, 1030, 1041, 1042, 1062, 1077, 1139, 1227, 1264, 1299, 1325, 1329, 1349, 1387, 1396, 1435, 1439, 1442, 1475, 1492, 1504, 1532, 1560, 1593, 1603, 1622, 1692, 1699, 1700, 1701, 1715, 1779, 1824, 1856, 1865, 1901, 1950, 1953, 1963, 1994, 1999, 2022, 2026, 2072, 2180, 2204, 2211, 2218, 2231, 2237, 2249, 2250, 2264, 2283, 2307, 2362, 2372, 2404, 2407, 2441, 2449, 2455, 2469, 2490, 2494, 2500, 2511, 2539, 2544, 2545, 2562, 2576, 2601, 2625, 2638, 2659, 2706, 2741, 2743, 2746, 2751, 2755, 2800, 2801, 2814, 2866, 2898, 2920, 2937, 2985, 2988, 2995, 3001, 3025, 3045, 3058, 3089, 3094, 3138, 3143, 3224, 3360, 3467, 3478, 3509, 3556, 3606, 3647, 3661, 3702, 3732, 3755, 3757, 3771, 3782, 3785, 3802, 3811, 3843, 3953, 4034, 4067, 4079, 4100, 4106, 4113, 4137, 4186, 4225, 4242, 4245, 4257, 4301, 43

In [58]:
print(point_to_update)
print(weights)


2250
[[ 2.  8.  4.  6.  2. 12.  3.  3.  2. 10.  1.  9.  2.  6.  2.  8.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 